<a href="https://colab.research.google.com/github/TrilokGuru/TCSRIO/blob/main/TCSRIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Create a dictionary to store each form ID and its writer
from google.colab import drive
import os
from itertools import islice
drive.mount("/content/gdrive",force_remount=True)
form_writer = {}
with open('gdrive/My Drive/TCS RIO/forms.txt') as f:
  for line in islice(f, 16, None):
    line_list = line.split(' ')
    form_id = line_list[0]
    writer = line_list[1]
    form_writer[form_id] = writer
list(form_writer.items())[0:5]


# Select the 50 most common writer
from collections import Counter
top_writers = []
num_writers = 50
writers_counter = Counter(form_writer.values())
for writer_id,_ in writers_counter.most_common(num_writers):
    top_writers.append(writer_id)
print(top_writers[0:5])


top_forms = []
for form_id, author_id in form_writer.items():
    if author_id in top_writers:
        top_forms.append(form_id)
print(top_forms[0:5])


import glob
import numpy as np
tmp = []
target_list = []
path_to_files = os.path.join('gdrive/My Drive/TCS RIO/temp_sentences', '*')
for filename in sorted(glob.glob(path_to_files)):
  tmp.append(filename)
  image_name = filename.split('/')[-1]
  file, ext = os.path.splitext(image_name)
  parts = file.split('-')
  form = parts[0] + '-' + parts[1]
  for key in form_writer:
    if key == form:
      target_list.append(str(form_writer[form]))
img_files = np.array(tmp)
img_targets = np.asarray(target_list)
print(img_files[0:5])
print(img_targets[0:5])
print(img_files.shape)
print(img_targets.shape)


import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
for file_name in img_files[:2]:
    img = mpimg.imread(file_name)
    plt.figure(figsize = (10,10))
    plt.imshow(img, cmap ='gray')


from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(img_targets)
encoded_img_targets = encoder.transform(img_targets)
print("Writer ID        : ", img_targets[:2])
print("Encoded writer ID: ", encoded_img_targets[:2])


from sklearn.model_selection import train_test_split
# Split dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(img_files, encoded_img_targets, test_size=0.2, shuffle = True)
# Further split training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle = True)
print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)


CROP_SIZE = 113
NUM_LABELS = 50
BATCH_SIZE = 16


#pip install pillow
from sklearn.utils import shuffle
from PIL import Image
import random
def get_augmented_sample(sample, label, sample_ratio):
    # Get current image details
    img = Image.open(sample)
    img_width = img.size[0]
    img_height = img.size[1]
    # Compute resize dimensions such that aspect ratio is maintained
    height_fac = CROP_SIZE / img_height
    size = (int(img_width * height_fac), CROP_SIZE)
    # Resize image 
    new_img = img.resize((size), Image.ANTIALIAS)
    new_img_width = new_img.size[0]
    new_img_height = new_img.size[1]
    # Generate a random number of crops of size 113x113 from the resized image
    x_coord = list(range(0, new_img_width - CROP_SIZE))
    num_crops = int(len(x_coord) * sample_ratio)
    random_x_coord = random.sample(x_coord, num_crops)
    # Create augmented images (cropped forms) and map them to a label (writer)
    images = []
    labels = []
    for x in random_x_coord:
        img_crop = new_img.crop((x, 0, x + CROP_SIZE, CROP_SIZE))
        # Transform image to an array of numbers
        images.append(np.asarray(img_crop))
        labels.append(label)
    return (images, labels)
sample, label = X_train[0], y_train[0]
img = mpimg.imread(sample)
plt.figure(figsize = (10,10))
plt.imshow(img, cmap ='gray')
print("Label: ", label)


images, labels = get_augmented_sample(sample, label, 0.1)
print(labels)
print("Num of labels: ", len(labels))


print(len(images))
plt.imshow(images[0], cmap ='gray');
plt.imshow(images[1], cmap ='gray');


import operator
from functools import reduce
from keras.utils import to_categorical
def generate_data(samples, labels, batch_size, sample_ratio):
    while 1: 
        for offset in range(0, len(samples), batch_size):
            batch_samples = samples[offset:(offset + batch_size)]
            batch_labels = labels[offset:(offset + batch_size)]     
            # Augment each sample in batch
            augmented_batch_samples = []
            augmented_batch_labels = []
            for i in range(len(batch_samples)):
                sample = batch_samples[i]
                label = batch_labels[i]
                augmented_samples, augmented_labels = get_augmented_sample(sample, label, sample_ratio)
                augmented_batch_samples.append(augmented_samples)
                augmented_batch_labels.append(augmented_labels)
            # Flatten out samples and labels
            augmented_batch_samples = reduce(operator.add, augmented_batch_samples)
            augmented_batch_labels = reduce(operator.add, augmented_batch_labels)           
            # Reshape input format
            X_train = np.array(augmented_batch_samples)
            X_train = X_train.reshape(X_train.shape[0], CROP_SIZE, CROP_SIZE, 1)
            # Transform input to float and normalize
            X_train = X_train.astype('float32')
            X_train /= 255
            # Encode y
            y_train = np.array(augmented_batch_labels)
            y_train = to_categorical(y_train, NUM_LABELS)
            yield X_train, y_train


train_generator = generate_data(X_train, y_train, BATCH_SIZE, 0.3)
validation_generator = generate_data(X_val, y_val, BATCH_SIZE, 0.3)
test_generator = generate_data(X_test, y_test, BATCH_SIZE, 0.1)


import tensorflow as tf
#config = tf.config.experimental
#tf.Session(config = config)
config = tf.compat.v1.ConfigProto()
tf.compat.v1.Session(config = config)


def resize_image(img):
    size = round(CROP_SIZE/2)
    return tf.image.resize(img, [size, size])


from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers.convolutional import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.optimizers import Adam
from keras import metrics
model = Sequential()
# Define network input shape
model.add(ZeroPadding2D((1, 1), input_shape=(CROP_SIZE, CROP_SIZE, 1)))
# Resize images to allow for easy computation
model.add(Lambda(resize_image)) 
# CNN model - Building the model suggested in paper
model.add(Convolution2D(filters= 32, kernel_size =(5,5), strides= (2, 2), padding='same', name='conv1'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool1'))
model.add(Convolution2D(filters= 64, kernel_size =(3, 3), strides= (1, 1), padding='same', name='conv2'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool2'))
model.add(Convolution2D(filters= 128, kernel_size =(3, 3), strides= (1, 1), padding='same', name='conv3'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool3'))


model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, name='dense1'))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256, name='dense2'))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_LABELS, name='output'))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['acc'])
print(model.summary())


from keras.callbacks import ModelCheckpoint
# Create directory to save checkpoints at
model_checkpoints_path = "gdrive/My Drive/TCS RIO/model_checkpoints"
if not os.path.exists(model_checkpoints_path):
    os.makedirs(model_checkpoints_path)
# Save model after every epoch using checkpoints
create_checkpoint = ModelCheckpoint(
    filepath = "gdrive/My Drive/TCS RIO/model_checkpoints/check-{epoch:02d}-{val_loss:.4f}.hdf5",
    verbose = 1,
    save_best_only = False
)
# Fit model using generators
history_object = model.fit_generator(
    train_generator, 
    steps_per_epoch = 700 #round(len(X_train) / BATCH_SIZE),
    validation_data = validation_generator,
    validation_steps = round(len(X_val) / BATCH_SIZE),
    epochs = 40,
    verbose = 1,
    callbacks = [create_checkpoint]
)


model_weights_path = "gdrive/My Drive/TCS RIO/model_checkpoints/check-30-0.2748.hdf5"
if model_weights_path:
    model.load_weights(model_weights_path)
    scores = model.evaluate_generator(test_generator, steps=round(len(X_test)/BATCH_SIZE))
    print("Accuracy: ", scores[1])
else:
    print("Set model weights file to load in the 'model_weights_path' variable")


# Load test data
from random import sample 
images = []
for filename in X_test[:50]:
  im = Image.open(filename)
  cur_width = im.size[0]
  cur_height = im.size[1]
  # print("Before Crop: ", cur_width, cur_height)
  height_fac = CROP_SIZE / cur_height
  new_width = int(cur_width * height_fac)
  size = new_width, CROP_SIZE
  # Resize so height = 113 while keeping aspect ratio
  imresize = im.resize((size), Image.ANTIALIAS)  
  now_width = imresize.size[0]
  now_height = imresize.size[1]
  # print("After Crop: ", now_width, now_height)
  # Generate crops of size 113x113 from this resized image and keep random 10% of crops
  avail_x_points = list(range(0, now_width - CROP_SIZE ))# total x start points are from 0 to width -113
  # Pick random x%
  factor = 0.1
  pick_num = int(len(avail_x_points)*factor)
  # print("Picked Number:", pick_num)
  random_startx = sample(avail_x_points,  pick_num)
  for start in random_startx:
      imcrop = imresize.crop((start, 0, start+CROP_SIZE, CROP_SIZE))
      images.append(np.asarray(imcrop))
  x_test = np.array(images)
  x_test = x_test.reshape(x_test.shape[0], CROP_SIZE, CROP_SIZE, 1)
  # convert to float and normalize
  x_test = x_test.astype('float32')
  x_test /= 255
  shuffle(x_test)
print(x_test.shape)


# Creating the predictive model
predictions = model.predict(x_test, verbose=1)
print(predictions.shape)
predicted_writer = []
for pred in predictions:
  predicted_writer.append(np.argmax(pred))
print(len(predicted_writer))


# Mapping Test image with Trained image

writer_number = 0
total_images = 10
counter = 0
for i in range(len(predicted_writer)//10):
  if predicted_writer[i] == writer_number:
    image = x_test[i].squeeze()
    plt.figure(figsize=(2,2))
    plt.imshow(image, cmap ='gray')
# print(x_test[1])
image1 = x_test[0].squeeze()
image2 = x_test[1].squeeze()
plt.figure(figsize=(2,2))
plt.imshow(image1, cmap='gray');
plt.figure(figsize=(2,2))
plt.imshow(image2, cmap='gray');
